In [102]:
import pandas as pd
import statsmodels.api as sm
import os

In [103]:
#ingresando datos
edmunds_data_new = pd.read_excel("edmunds_data.xlsx")

In [104]:

# Clasificando los vehículos en eléctricos y no eléctricos (convencionales)
vehiculos_electricos_new = edmunds_data_new[edmunds_data_new['Fuel'] == 'Electric']
vehiculos_no_electricos_new = edmunds_data_new[edmunds_data_new['Fuel'] != 'Electric']

# Resumen estadístico para vehículos eléctricos
resumen_electricos_new = vehiculos_electricos_new.describe()

# Resumen estadístico para vehículos no eléctricos (convencionales)
resumen_no_electricos_new = vehiculos_no_electricos_new.describe()

(resumen_electricos_new, resumen_no_electricos_new)


(         Year    City MPG  CO2 Emissions (g/mi)  Sound Emission (dB)  \
 count    12.0   12.000000                  12.0            12.000000   
 mean   2023.0   92.500000                   0.0            66.583333   
 std       0.0   34.568772                   0.0             1.164500   
 min    2023.0   23.000000                   0.0            65.000000   
 25%    2023.0   87.750000                   0.0            65.750000   
 50%    2023.0  104.000000                   0.0            67.000000   
 75%    2023.0  115.750000                   0.0            67.250000   
 max    2023.0  120.000000                   0.0            68.000000   
 
        Base Price (USD)  Range (mi)  
 count         12.000000   12.000000  
 mean       40379.916667  264.083333  
 std         6160.983252   55.688188  
 min        31000.000000  140.000000  
 25%        38230.500000  245.000000  
 50%        40970.000000  254.500000  
 75%        42213.750000  303.000000  
 max        51990.000000  358

- Vehículos Eléctricos
    - Eficiencia de Combustible (City MPG): Promedio de 92.5
    - Emisiones de CO2 (g/mi): 0 (no emiten CO2 durante la operación)
    - Emisiones de Sonido (dB): Promedio de 66.58
    - Precio Base (USD): Promedio de $40,379.92
    - Rango de Autonomía (mi): Promedio de 264.08

- Vehículos No Eléctricos (Convencionales)
    - Eficiencia de Combustible (City MPG): Promedio de 30.58
    - Emisiones de CO2 (g/mi): Promedio de 97.33
    - Emisiones de Sonido (dB): Promedio de 70.12
    - Precio Base (USD): Promedio de $32,315.00
    - Rango de Autonomía (mi): Promedio de 378.27

- Análisis Comparativo
    - Eficiencia de Combustible: Los vehículos eléctricos son mucho más eficientes en términos de MPG.3 veces mas
    - Emisiones de CO2: Los vehículos eléctricos tienen la ventaja de no emitir CO2 durante la operación.
    - Emisiones de Sonido: Los vehículos eléctricos tienden a ser más silenciosos.
    - Precio Base: En promedio, los vehículos eléctricos son más costosos.
    - Rango de Autonomía: Los vehículos convencionales tienen un mayor rango de autonomía.

modelo

In [105]:
import pandas as pd
import statsmodels.api as sm

# Cargar los datos desde el archivo 'edmunds_data.xlsx'
data = pd.read_excel('edmunds_data.xlsx')

# Crear una columna binaria para indicar si el vehículo es eléctrico o no
data['Es_Electrico'] = (data['Fuel'] == 'Electric').astype(int)

# Unificar las categorías 'Sedan' y 'Sedán'
data['Vehicle Type'] = data['Vehicle Type'].replace({'Sedán': 'Sedan'})

# Crear columnas binarias para el tipo de vehículo
vehicle_types = ['Minivan', 'Pick-up', 'SUV', 'Sedan']
for v_type in vehicle_types:
    data[f'Vehicle Type_{v_type}'] = (data['Vehicle Type'] == v_type).astype(int)

# Preparar los datos para el modelo de regresión
X = data[['Es_Electrico', 'Year', 'Base Price (USD)', 'Range (mi)'] + [f'Vehicle Type_{v_type}' for v_type in vehicle_types]]
X = sm.add_constant(X)  # Agregar constante

# Variables dependientes
y_emisiones = data['CO2 Emissions (g/mi)']
y_ruido = data['Sound Emission (dB)']

# Crear y ajustar los modelos de regresión con errores estándar robustos para statsmodels
modelo_emisiones_stats_robust = sm.OLS(y_emisiones, X).fit(cov_type='HC3')
modelo_ruido_stats_robust = sm.OLS(y_ruido, X).fit(cov_type='HC3')

# Resúmenes de los modelos con errores estándar robustos
resumen_modelo_emisiones_robust = modelo_emisiones_stats_robust.summary()
resumen_modelo_ruido_robust = modelo_ruido_stats_robust.summary()

# Imprimir resúmenes de estadísticas detalladas
print("Resumen de estadísticas para el modelo de emisiones de CO2 con errores estándar robustos:")
print(resumen_modelo_emisiones_robust)

print("\nResumen de estadísticas para el modelo de nivel de ruido con errores estándar robustos:")
print(resumen_modelo_ruido_robust)



Resumen de estadísticas para el modelo de emisiones de CO2 con errores estándar robustos:
                             OLS Regression Results                             
Dep. Variable:     CO2 Emissions (g/mi)   R-squared:                       0.588
Model:                              OLS   Adj. R-squared:                  0.552
Method:                   Least Squares   F-statistic:                     21.52
Date:                  Wed, 17 Jan 2024   Prob (F-statistic):           4.45e-18
Time:                          01:50:13   Log-Likelihood:                -546.67
No. Observations:                   101   AIC:                             1111.
Df Residuals:                        92   BIC:                             1135.
Df Model:                             8                                         
Covariance Type:                    HC3                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
-----------


- **Modelo de Emisiones de CO2:**
- R-squared (0.588): El modelo explica aproximadamente el 58.8% de la variabilidad en las emisiones de CO2. Es una medida razonable de ajuste del modelo.
- Coeficientes:
    -Es_Electrico: El coeficiente es -105.8704, significativo al nivel 0.000. Esto implica que, manteniendo todo lo demás constante, los vehículos eléctricos emiten en promedio 105.87 gramos menos de CO2 por milla que los vehículos no eléctricos.
    - Year: Cada año adicional está asociado con un aumento de 33.20 gramos de CO2 por milla, aunque este resultado es apenas significativo (P>|t|=0.045).
    - Base Price (USD): El impacto del precio base no es estadísticamente significativo.
    - Range (mi): El impacto del rango de autonomía no es estadísticamente significativo.
    - Vehicle Type: Solo el tipo 'Sedan' muestra un efecto significativo, sugiriendo que los sedanes emiten más CO2 que otros tipos de vehículos, todo lo demás constante.

- **Modelo de Nivel de Ruido:**
- R-squared (0.468): El modelo explica aproximadamente el 46.8% de la variabilidad en el nivel de ruido.
- Coeficientes:
    -Es_Electrico: El coeficiente es -1.301, significativo al nivel 0.015. Los vehículos eléctricos son en promedio 1.3 decibelios más silenciosos que los no eléctricos.
    - Year: El año del modelo no tiene un impacto significativo en el nivel de ruido.
    - Base Price (USD): Un precio base más alto está asociado con un nivel de ruido más bajo, significativo al nivel 0.000.
    - Range (mi): El rango de autonomía no tiene un impacto significativo en el nivel de ruido.
    - Vehicle Type: Los tipos 'Pick-up', 'SUV' y 'Sedan' muestran efectos significativos en el nivel de ruido.

In [106]:
# Función para crear un DataFrame a partir de los resultados del modelo

# Crear directorio si no existe
output_dir = 'Bases_api/modelo_electricoVsgasolina_edmunds'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
def model_results_to_df(model_results):
    # Extrayendo los coeficientes, errores estándar y p-values
    params = model_results.params
    std_err = model_results.bse
    p_values = model_results.pvalues

    # Creando el DataFrame
    df = pd.DataFrame({
        'Coef': params,
        'Std Err': std_err,
        'P>|t|': p_values
    })

    # Agregando una columna para indicar si es significativo
    df['Significativo'] = df['P>|t|'] < 0.05

    return df

# Crear DataFrames para los modelos de emisiones y ruido
df_emisiones = model_results_to_df(modelo_emisiones_stats)
df_ruido = model_results_to_df(modelo_ruido_stats)

# Rutas para guardar los archivos CSV
emisiones_csv_path = f'{output_dir}/regresion_edmunds_emisiones.csv'
ruido_csv_path = f'{output_dir}/regresion_edmunds_ruido.csv'

# Guardar los DataFrames en archivos CSV
df_emisiones.to_csv(emisiones_csv_path)
df_ruido.to_csv(ruido_csv_path)

emisiones_csv_path, ruido_csv_path


('Bases_api/modelo_electricoVsgasolina_edmunds/regresion_edmunds_emisiones.csv',
 'Bases_api/modelo_electricoVsgasolina_edmunds/regresion_edmunds_ruido.csv')